In [1]:
import requests
import fake_useragent
from bs4 import BeautifulSoup

In [2]:
ua = fake_useragent.UserAgent()
ua_chrome = ua.chrome
print(ua_chrome)

Mozilla/5.0 (Windows; U; Windows NT 6.0; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/4.0.211.7 Safari/532.0


In [3]:
url = "https://www.unternehmensregister.de/ureg/"

payload = {}

headers = {
  'Host': 'www.unternehmensregister.de',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
  'User-Agent': ua_chrome,
  'Accept-Language': 'en-GB,en;q=0.9'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)

<!DOCTYPE html>
<html lang="de" xmlns="http://www.w3.org/1999/xhtml"><head id="j_idt2">
    <title>Unternehmensregister</title>
    <meta charset="UTF-8" />
    <link rel="stylesheet" href="assets/css/style.min.css" />
    <!-- link rel="stylesheet" type="text/css" href="/ureg/styles/regport.css" media="screen"/ -->
    <link rel="stylesheet" type="text/css" href="/ureg/styles/main-redesign.css" media="screen" />
    <link rel="stylesheet" type="text/css" href="/ureg/styles/cc.css" media="screen" />
    <link rel="stylesheet" type="text/css" href="/ureg/styles/plus.css" media="screen" />

    <script type="text/javascript">
          var tooltip_str_print = "Drucken";
          var tooltip_str_close = "Schließen";
          var tooltip_str_image = "Bild";
          var tooltip_str_of    = "von";
          var tooltip_str_next  = "Nächstes";
          var tooltip_str_prev  = "Vorheriges";
          var tooltip_str_loading = "Bilddaten werden geladen …";
          var tooltip_str_error =

In [4]:
# read token and javax state
import re

regex = r'jsessionid=([^"]+)'

match = re.search(regex, response.text)
if match:
    jsessionid = match.group(1)
    print(jsessionid)
    

soup = BeautifulSoup(response.text, 'html.parser')

input_element = soup.find('input', {'name': 'javax.faces.ViewState'})
if input_element:
    viewstate = input_element['value']
    print(viewstate)

84E94D6BF6B989C5B3253CB972172EB0.web04-1?submitaction=closeBrowserBanner
-4646064976215944631:-282274654117584858


In [5]:
import requests

search_term = "Tanso"
url = f"https://www.unternehmensregister.de//ureg/index.html;jsessionid={jsessionid}"

payload = f'globalSearchForm=globalSearchForm&globalSearchForm%3AextendedResearchCompanyName={search_term}&globalSearchForm%3Ahidden_element_used_for_validation_only=aaa&submitaction=globalsearch&globalsearch=true&globalSearchForm%3AbtnExecuteSearchOld=Suchen&javax.faces.ViewState={viewstate}'
headers = {
  'Host': 'www.unternehmensregister.de',
  'Origin': 'https://www.unternehmensregister.de',
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
  'User-Agent': ua_chrome,
  'Referer': 'https://www.unternehmensregister.de/ureg/',
  'Accept-Language': 'en-GB,en;q=0.9',
  'Content-Type': 'application/x-www-form-urlencoded'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)


<!DOCTYPE html>
<html lang="de" xmlns="http://www.w3.org/1999/xhtml"><head id="j_idt2">
    <title>Unternehmensregister</title>
    <meta charset="UTF-8" />
    <link rel="stylesheet" href="assets/css/style.min.css" />
    <!-- link rel="stylesheet" type="text/css" href="/ureg/styles/regport.css" media="screen"/ -->
    <link rel="stylesheet" type="text/css" href="/ureg/styles/main-redesign.css" media="screen" />
    <link rel="stylesheet" type="text/css" href="/ureg/styles/cc.css" media="screen" />
    <link rel="stylesheet" type="text/css" href="/ureg/styles/plus.css" media="screen" />

    <script type="text/javascript">
          var tooltip_str_print = "Drucken";
          var tooltip_str_close = "Schließen";
          var tooltip_str_image = "Bild";
          var tooltip_str_of    = "von";
          var tooltip_str_next  = "Nächstes";
          var tooltip_str_prev  = "Vorheriges";
          var tooltip_str_loading = "Bilddaten werden geladen …";
          var tooltip_str_error =